单边稀疏


![alt text](image.png)

In [ ]:
sparse_IO = [
    {'pc': 0x1ABC, 'A_indptr': 0, 'A_indices': 2, 'A_address': 1000, 'destination_register': 1},
    {'pc': 0x1ABC, 'A_indptr': 1, 'A_indices': 4, 'A_address': 1016, 'destination_register': 2},
    {'pc': 0x1ABC, 'A_indptr': 1, 'A_indices': 6, 'A_address': 1032, 'destination_register': 3},
    {'pc': 0x1ABC, 'A_indptr': 2, 'A_indices': 2, 'A_address': 1048, 'destination_register': 4},
    {'pc': 0x1ABC, 'B_address': 1048, 'destination_register': 4},
    {'pc': 0x1ABC, 'B_address': 1048, 'destination_register': 4},
    {'pc': 0x1ABC, 'B_address': 1048, 'destination_register': 4},
    {'pc': 0x1ABC, 'B_address': 1048, 'destination_register': 4},
]


TaintTracker (TT) module: 
- TT entry
每个TT表项由以下组成:
- Register ID（寄存器ID）: 8比特。标记寄存器的编号。
- Tainted（是否被污染）: 1比特。表示该寄存器是否被污染（被间接加载链影响）。
- Speculative Register ID（SRF寄存器ID）: 8比特。用于指向在Speculative Register File中的位置。
- Offset（偏移）: 8比特。记录被污染的指令到起始间接加载的偏移量，用于替换时的LRU策略。
- Mapped（是否被映射到SRF）: 1比特。标记该寄存器是否被映射到SRF寄存器中。

In [12]:
class TaintTrackerEntry:
    def __init__(self, register_id):
        self.register_id = register_id  #### 8-bit 寄存器ID
        self.tainted = 0  ###### 1-bit 是否被污染
        self.srf_id = 0  ###### 8-bit SRF寄存器ID
        self.offset = 0  ###### 8-bit 偏移
        self.mapped = 0  ###### 1-bit 是否被映射

    def __repr__(self):
        return (f"RegID: {self.register_id}, Tainted: {self.tainted}, SRF_ID: {self.srf_id}, "
                f"Offset: {self.offset}, Mapped: {self.mapped}")


class TaintTracker:
    def __init__(self, max_entries=16):
        self.entries = {}  ##### 存储所有的寄存器条目，按寄存器ID索引
        self.max_entries = max_entries  ###### 最大条目数
        self.access_order = []  ###### 记录访问顺序，用于LRU策略

    def _update_access_order(self, register_id):
        """ 更新访问顺序，移到最后表示最近使用 """
        if register_id in self.access_order:
            self.access_order.remove(register_id)
        self.access_order.append(register_id)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_register = self.access_order.pop(0)
            del self.entries[oldest_register]
            print(f"Evicted Register {oldest_register} due to capacity limit.")

    def taint_register(self, register_id, srf_id, offset):
        """ 将指定寄存器标记为污染状态，并映射到SRF """
        if register_id not in self.entries:
            ###### 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            ###### 创建新条目
            self.entries[register_id] = TaintTrackerEntry(register_id)
        
        entry = self.entries[register_id]
        entry.tainted = 1  ###### 标记寄存器被污染
        entry.srf_id = srf_id  ###### 映射到的SRF ID
        entry.offset = offset  ###### 设置偏移
        entry.mapped = 1  ###### 设置映射标记

        ###### 更新访问顺序
        self._update_access_order(register_id)

    def clear_taint(self, register_id):
        """ 清除指定寄存器的污染状态 """
        if register_id in self.entries:
            entry = self.entries[register_id]
            entry.tainted = 0
            entry.mapped = 0  ###### 清除映射标记

            ###### 更新访问顺序（清除污染状态后仍视为最近访问）
            self._update_access_order(register_id)

    def is_tainted(self, register_id):
        """ 检查指定寄存器是否被污染 """
        if register_id in self.entries:
            return self.entries[register_id].tainted == 1
        return False

    def get_entry(self, register_id):
        """ 返回指定寄存器的条目 """
        if register_id in self.entries:
            return self.entries[register_id]
        return None

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Taint Tracker Entries (Max Entries: {self.max_entries}):\n{entries_repr}"

    
if __name__ == "__main__":
    ###### 初始化Taint Tracker，设置最大条目数为32
    tt = TaintTracker(max_entries=4)

    ###### 测试数据：超过最大条目数的寄存器ID
    register_ids = [1, 2, 3, 4, 5]  ###### 测试的寄存器ID（4个条目）
    srf_ids = [100, 101, 102, 103, 104]  ###### 对应的SRF ID
    offsets = [10, 20, 30, 40, 50]  ###### 设置每个寄存器的偏移量

    print("Initial State:")
    print(tt)

    ###### 标记寄存器为污染状态，超过最大条目数后应触发替换
    for reg_id, srf_id, offset in zip(register_ids, srf_ids, offsets):
        tt.taint_register(reg_id, srf_id, offset)
        print(f"Tainted Register {reg_id} -> SRF ID: {srf_id}, Offset: {offset}")
        print(tt)

    ###### 打印最终的Taint Tracker状态
    print("Final State:")
    print(tt)



Initial State:
Taint Tracker Entries (Max Entries: 4):

Tainted Register 1 -> SRF ID: 100, Offset: 10
Taint Tracker Entries (Max Entries: 4):
RegID: 1, Tainted: 1, SRF_ID: 100, Offset: 10, Mapped: 1
Tainted Register 2 -> SRF ID: 101, Offset: 20
Taint Tracker Entries (Max Entries: 4):
RegID: 1, Tainted: 1, SRF_ID: 100, Offset: 10, Mapped: 1
RegID: 2, Tainted: 1, SRF_ID: 101, Offset: 20, Mapped: 1
Tainted Register 3 -> SRF ID: 102, Offset: 30
Taint Tracker Entries (Max Entries: 4):
RegID: 1, Tainted: 1, SRF_ID: 100, Offset: 10, Mapped: 1
RegID: 2, Tainted: 1, SRF_ID: 101, Offset: 20, Mapped: 1
RegID: 3, Tainted: 1, SRF_ID: 102, Offset: 30, Mapped: 1
Tainted Register 4 -> SRF ID: 103, Offset: 40
Taint Tracker Entries (Max Entries: 4):
RegID: 1, Tainted: 1, SRF_ID: 100, Offset: 10, Mapped: 1
RegID: 2, Tainted: 1, SRF_ID: 101, Offset: 20, Mapped: 1
RegID: 3, Tainted: 1, SRF_ID: 102, Offset: 30, Mapped: 1
RegID: 4, Tainted: 1, SRF_ID: 103, Offset: 40, Mapped: 1
Evicted Register 1 due to capa

Stride Detector(SD), 包含以下字段：
- PC（程序计数器）: 48比特。用于记录加载指令的地址，作为唯一标识。
- Previous Address（前一个地址）: 48比特。存储该指令上一次的访问地址。
- Stride（步长）: 8比特。存储当前的步长值，用于判断步长模式。
- Last Prefetch（上次预取地址）: 48比特。记录上次生成的预取地址，用于防止重复预取。
- Seen（是否检测到）: 1比特。标记该指令是否已经检测到步长模式。
- LIL（最后一个间接加载）: 16比特。记录最后一个间接加载的地址偏移量。
- Stride Confidence（步长置信度）: 2比特。表示步长模式检测的置信度（如：0表示未检测到，1表示低置信度，2表示中置信度，3表示高置信度）。
- LIL Confidence（LIL置信度）: 2比特。表示最后一个间接加载检测的置信度。

LBD（Loop Bound Detector）结构设计
Loop Bound Detector (LBD) 主要用于检测循环的边界和条件，并预测循环的迭代次数。其设计包括以下字段：

- PC（程序计数器）: 48 比特。用于记录循环指令的地址。
- Comp-PC（比较指令PC）: 48 比特。用于记录上一次循环比较指令的PC值。
- EWMA（指数加权移动平均）: 9 比特。用于记录循环的平均迭代次数，便于预测下次的迭代次数。
- Iteration Counter（迭代计数器）: 9 比特。记录当前循环的迭代次数。
- Loop Increment（循环增量）: 16 比特。记录每次循环迭代的增量（例如，i += 2 的增量为 2）。
- Confidence（置信度）: 2 比特。表示循环边界预测的置信度。
- Source Value A/B（源值A/B）: 64x2 比特。用于存储上一次循环比较指令中的两个源操作数值，用于预测循环边界。
- Register ID A/B（寄存器ID）: 5x2 比特。记录当前循环的寄存器ID，用于存储循环的迭代次数。

In [14]:
class LoopBoundEntry:
    def __init__(self, pc):
        self.pc = pc  ###### 48-bit 程序计数器
        self.comp_pc = 0  ###### 48-bit 比较指令的PC值
        self.ewma = 0  ###### 9-bit 指数加权移动平均
        self.iteration_counter = 0  ###### 9-bit 迭代计数器
        self.loop_increment = 0  ###### 16-bit 循环增量
        self.confidence = 0  ###### 2-bit 置信度
        self.source_val_a = 0  ###### 64-bit 源值A
        self.source_val_b = 0  ###### 64-bit 源值B
        self.reg_id_a = 0  ###### 5-bit 寄存器ID A
        self.reg_id_b = 0  ###### 5-bit 寄存器ID B

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, Comp-PC: {self.comp_pc}, EWMA: {self.ewma}, "
                f"Iteration: {self.iteration_counter}, Increment: {self.loop_increment}, "
                f"Confidence: {self.confidence}, SrcA: {self.source_val_a}, SrcB: {self.source_val_b}, "
                f"RegID_A: {self.reg_id_a}, RegID_B: {self.reg_id_b}")


class LoopBoundDetector:
    def __init__(self, max_entries=16):
        self.entries = {}  ###### 存储所有的循环条目，按PC索引
        self.max_entries = max_entries  ###### 最大条目数
        self.access_order = []  ###### 记录访问顺序，用于LRU替换策略

    def _update_access_order(self, pc):
        """ 更新访问顺序，移到最后表示最近使用 """
        if pc in self.access_order:
            self.access_order.remove(pc)
        self.access_order.append(pc)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_pc = self.access_order.pop(0)  ###### 移除最久未使用的条目
            del self.entries[oldest_pc]
            print(f"Evicted Loop Entry for PC {oldest_pc} due to capacity limit.")

    def update_loop_bound(self, pc, comp_pc, ewma, iteration, increment, src_a, src_b, reg_id_a, reg_id_b):
        """ 更新指定PC的循环边界信息，增加寄存器ID """
        if pc not in self.entries:
            ###### 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            ###### 创建新的Loop Bound条目
            self.entries[pc] = LoopBoundEntry(pc)
        
        entry = self.entries[pc]
        entry.comp_pc = comp_pc
        entry.ewma = ewma
        entry.iteration_counter = iteration
        entry.loop_increment = increment
        entry.source_val_a = src_a
        entry.source_val_b = src_b
        entry.reg_id_a = reg_id_a  ###### 设置寄存器ID A
        entry.reg_id_b = reg_id_b  ###### 设置寄存器ID B

        ###### 根据迭代次数和EWMA更新置信度
        entry.confidence = min(3, entry.confidence + 1 if iteration <= ewma else entry.confidence - 1)

        ###### 更新访问顺序
        self._update_access_order(pc)

    def clear_entry(self, pc):
        """ 清除指定PC的循环条目 """
        if pc in self.entries:
            del self.entries[pc]
            print(f"Cleared Loop Entry for PC {hex(pc)}")
            if pc in self.access_order:
                self.access_order.remove(pc)

    def get_entry(self, pc):
        """ 返回指定PC的条目 """
        if pc in self.entries:
            return self.entries[pc]
        return None

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Loop Bound Detector Entries (Max Entries: {self.max_entries}):\n{entries_repr}"

if __name__ == "__main__":
    lbd = LoopBoundDetector(max_entries=3)

    ###### 测试数据（包括寄存器ID，超过最大条目数）
    test_cases = [
        (0x1, 0x10, 5, 5, 2, 100, 200, 1, 2),  
        ###### (PC, Comp-PC, EWMA, Iteration, Increment, SrcA, SrcB, RegID_A, RegID_B)
        (0x2, 0x20, 10, 12, 2, 110, 210, 3, 4),
        (0x3, 0x30, 15, 14, 1, 120, 220, 5, 6),
        (0x4, 0x40, 20, 18, 3, 130, 230, 7, 8),  ###### 超过最大条目数，触发LRU替换
        (0x5, 0x50, 8, 8, 2, 140, 240, 9, 10),
    ]

    print("Initial State:")
    print(lbd)

    ###### 对每个测试用例进行循环边界更新
    for pc, comp_pc, ewma, iteration, increment, src_a, src_b, reg_id_a, reg_id_b in test_cases:
        lbd.update_loop_bound(pc, comp_pc, ewma, iteration, increment, src_a, src_b, reg_id_a, reg_id_b)
        print(f"Updated Loop Bound for PC: {hex(pc)}")
        print(lbd)

    ###### 打印最终的 Loop Bound Detector 状态
    print("Final State:")
    print(lbd)


Initial State:
Loop Bound Detector Entries (Max Entries: 3):

Updated Loop Bound for PC: 1
Loop Bound Detector Entries (Max Entries: 3):
PC: 1, Comp-PC: 16, EWMA: 5, Iteration: 5, Increment: 2, Confidence: 1, SrcA: 100, SrcB: 200, RegID_A: 1, RegID_B: 2
Updated Loop Bound for PC: 2
Loop Bound Detector Entries (Max Entries: 3):
PC: 1, Comp-PC: 16, EWMA: 5, Iteration: 5, Increment: 2, Confidence: 1, SrcA: 100, SrcB: 200, RegID_A: 1, RegID_B: 2
PC: 2, Comp-PC: 32, EWMA: 10, Iteration: 12, Increment: 2, Confidence: -1, SrcA: 110, SrcB: 210, RegID_A: 3, RegID_B: 4
Updated Loop Bound for PC: 3
Loop Bound Detector Entries (Max Entries: 3):
PC: 1, Comp-PC: 16, EWMA: 5, Iteration: 5, Increment: 2, Confidence: 1, SrcA: 100, SrcB: 200, RegID_A: 1, RegID_B: 2
PC: 2, Comp-PC: 32, EWMA: 10, Iteration: 12, Increment: 2, Confidence: -1, SrcA: 110, SrcB: 210, RegID_A: 3, RegID_B: 4
PC: 3, Comp-PC: 48, EWMA: 15, Iteration: 14, Increment: 1, Confidence: 1, SrcA: 120, SrcB: 220, RegID_A: 5, RegID_B: 6
Evi

HSLR 用于保存加载指令的 PC 和控制流掩码，主要由以下字段组成：

- PC（程序计数器）: 48 比特。用于记录当前步长加载指令的地址。
- Mask（控制流掩码）: 16 比特。用于管理控制流的分歧，特别是对分支指令的路径选择进行记录和管理。

In [15]:
class HSLR:
    def __init__(self):
        self.pc = 0  ###### 48-bit 程序计数器
        self.mask = 0  ###### 16-bit 控制流掩码

    def write(self, pc, mask):
        """ 更新HSLR寄存器的内容 """
        self.pc = pc
        self.mask = mask

    def __repr__(self):
        return f"HSLR Register: PC: {hex(self.pc)}, Mask: {self.mask}"

if __name__ == "__main__":
    hslr = HSLR()

    ###### 更新HSLR寄存器
    hslr.write(pc=0x1ABCDE, mask=0xFFFF)
    print(hslr)

    ###### 更新HSLR寄存器为另一个值
    hslr.write(0x123456, 0xAAAA)
    print(hslr)



HSLR Register: PC: 1752286, Mask: 65535
HSLR Register: PC: 1193046, Mask: 43690


Scoreboard:
- PC（程序计数器）: 48 比特。用于记录对应指令的地址。
- Status（状态标志位）: 5 比特。用于标识指令的状态（例如：未发出、正在执行、已完成等）。
- Return Counter（返回计数器）(新添加): ⌈log2(N + 1)⌉ 比特。用于记录正在执行的标量指令数量。每个条目对应的标量指令数 N 在初始化时设定。

In [16]:
import math

class ScoreboardEntry:
    def __init__(self, pc, status=0, return_counter=0, max_scalars=16):
        self.pc = pc  ###### 48-bit 程序计数器
        self.status = status  ###### 5-bit 状态标志位，表示指令状态
        self.return_counter = return_counter  ###### 返回计数器，初始为 0
        self.max_scalars = max_scalars  ###### 最大标量指令数，用于设置计数器的最大值

        ###### 计算 log2(N + 1) 的比特数，表示返回计数器的比特位宽
        self.return_counter_bits = math.ceil(math.log2(max_scalars + 1))

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, Status: {self.status}, "
                f"Return Counter: {self.return_counter} / {self.max_scalars} (Bits: {self.return_counter_bits})")

    def update_status(self, new_status):
        """ 更新指令的状态 """
        self.status = new_status

    def increment_return_counter(self, increment=1):
        """ 增加返回计数器值，表示发出了更多的标量指令 """
        self.return_counter = min(self.max_scalars, self.return_counter + increment)

    def decrement_return_counter(self, decrement=1):
        """ 更新返回计数器，指令执行完毕时递减计数器 """
        self.return_counter = max(0, self.return_counter - decrement)  ###### 确保计数器不为负值

    def is_completed(self):
        """ 判断所有标量指令是否已经执行完毕 """
        return self.return_counter == 0


class Scoreboard:
    def __init__(self, max_entries=32, max_scalars=16):
        self.entries = {}  ###### 存储所有的Scoreboard条目，按PC索引
        self.max_entries = max_entries  ###### 最大条目数
        self.max_scalars = max_scalars  ###### 每个条目中标量指令的最大数量
        self.access_order = []  ###### 记录访问顺序，用于LRU替换策略

    def _update_access_order(self, pc):
        """ 更新访问顺序，移到最后表示最近使用 """
        if pc in self.access_order:
            self.access_order.remove(pc)
        self.access_order.append(pc)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_pc = self.access_order.pop(0)  ###### 移除最久未使用的条目
            del self.entries[oldest_pc]
            print(f"Evicted Scoreboard Entry for PC {oldest_pc} due to capacity limit.")

    def add_entry(self, pc, status=0):
        """ 添加新的Scoreboard条目，并初始化返回计数器 """
        if pc not in self.entries:
            ###### 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            ###### 创建新的 Scoreboard 条目，并将返回计数器初始化为标量指令数量
            self.entries[pc] = ScoreboardEntry(pc, status, return_counter=self.max_scalars, max_scalars=self.max_scalars)
            print(f"Added Scoreboard Entry for PC {hex(pc)} with Return Counter set to {self.max_scalars}")

        ###### 更新访问顺序
        self._update_access_order(pc)

    def update_entry_status(self, pc, new_status):
        """ 更新指定PC的状态 """
        if pc in self.entries:
            self.entries[pc].update_status(new_status)
            print(f"Updated Status for PC {hex(pc)} to {new_status}")

    def decrement_return_counter(self, pc, decrement=1):
        """ 更新指定PC的返回计数器 """
        if pc in self.entries:
            self.entries[pc].decrement_return_counter(decrement)
            print(f"Updated Return Counter for PC {hex(pc)} by decrementing {decrement}")
            ###### 检查返回计数器是否已归零
            if self.entries[pc].is_completed():
                print(f"All scalar instructions for PC {hex(pc)} have completed.")

    def get_entry(self, pc):
        """ 返回指定PC的条目 """
        if pc in self.entries:
            return self.entries[pc]
        return None

    def clear_entry(self, pc):
        """ 清除指定PC的Scoreboard条目 """
        if pc in self.entries:
            del self.entries[pc]
            print(f"Cleared Scoreboard Entry for PC {hex(pc)}")
            if pc in self.access_order:
                self.access_order.remove(pc)

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Scoreboard Entries (Max Entries: {self.max_entries}):\n{entries_repr}"
    
if __name__ == "__main__":    
    sb = Scoreboard(max_entries=4, max_scalars=16)

    ###### 添加4个Scoreboard条目
    sb.add_entry(0x1, status=1)
    sb.add_entry(0x2, status=2)
    sb.add_entry(0x3, status=3)
    sb.add_entry(0x4, status=0)
    print("Initial Scoreboard State:")
    print(sb)

    ###### 更新某个条目的状态和返回计数器
    sb.update_entry_status(0x1, new_status=5)
    sb.decrement_return_counter(0x1, decrement=1)  ###### 减少返回计数器
    sb.decrement_return_counter(0x2, decrement=15)  ###### 返回计数器归零

    ###### 再添加一个条目，触发LRU替换
    sb.add_entry(0x5, status=0)

    print("Final Scoreboard State:")
    print(sb)


Added Scoreboard Entry for PC 1 with Return Counter set to 16
Added Scoreboard Entry for PC 2 with Return Counter set to 16
Added Scoreboard Entry for PC 3 with Return Counter set to 16
Added Scoreboard Entry for PC 4 with Return Counter set to 16
Initial Scoreboard State:
Scoreboard Entries (Max Entries: 4):
PC: 1, Status: 1, Return Counter: 16 / 16 (Bits: 5)
PC: 2, Status: 2, Return Counter: 16 / 16 (Bits: 5)
PC: 3, Status: 3, Return Counter: 16 / 16 (Bits: 5)
PC: 4, Status: 0, Return Counter: 16 / 16 (Bits: 5)
Updated Status for PC 1 to 5
Updated Return Counter for PC 1 by decrementing 1
Updated Return Counter for PC 2 by decrementing 15
Evicted Scoreboard Entry for PC 1 due to capacity limit.
Added Scoreboard Entry for PC 5 with Return Counter set to 16
Final Scoreboard State:
Scoreboard Entries (Max Entries: 4):
PC: 2, Status: 2, Return Counter: 1 / 16 (Bits: 5)
PC: 3, Status: 3, Return Counter: 16 / 16 (Bits: 5)
PC: 4, Status: 0, Return Counter: 16 / 16 (Bits: 5)
PC: 5, Status: 0

SVU:生成SVI向量化指令

In [17]:
class ScalarVectorUnit:
    def __init__(self, scalar_vector_length=16):
        ###### Scalar Vector Unit 的配置项
        self.scalar_vector_length = scalar_vector_length  ###### 定义 SVU 中标量拷贝的数量
        self.srf = {}  ###### Speculative Register File (SRF)，存储标量拷贝的执行结果
        self.taunt_tracker = {}  ###### 污染追踪器，用于标记被污染的寄存器
        
    def generate_svi(self, pc, base_address, stride):
        """ 根据基地址和步长生成 Scalar Vector Instructions (SVIs) """
        svi_list = []
        for i in range(self.scalar_vector_length):
            ###### 每个 SVI 是一个标量指令的拷贝，地址为 base_address + i * stride
            new_address = base_address + i * stride
            svi_list.append((pc, new_address))
        return svi_list

    def execute_svi(self, svi_list):
        """ 模拟执行生成的 SVIs，并将结果写入推测寄存器文件 (SRF) """
        for (pc, addr) in svi_list:
            ###### 模拟加载数据：将地址值存入 SRF 中，表示已加载该地址
            self.srf[pc] = addr
            print(f"Executed SVI for PC: {hex(pc)} at address: {addr}")

    def update_taint_tracker(self, reg_id):
        """ 更新污染追踪器，标记某个寄存器已经被污染 """
        self.taunt_tracker[reg_id] = True
        print(f"Register {reg_id} marked as tainted.")

    def clear_taint(self, reg_id):
        """ 清除某个寄存器的污染标记 """
        if reg_id in self.taunt_tracker:
            del self.taunt_tracker[reg_id]
            print(f"Taint cleared for Register {reg_id}.")

    def __repr__(self):
        srf_repr = "\n".join([f"PC: {hex(pc)}, Data: {data}" for pc, data in self.srf.items()])
        return f"Scalar-Vector Unit (SVU) State:\nSRF Entries:\n{srf_repr}\nTaint Tracker: {self.taunt_tracker}"


if __name__ == "__main__":
    svu = ScalarVectorUnit(scalar_vector_length=4)

    ###### 假设一个步长加载指令 PC 为 0x1ABC，基地址为 1000，步长为 16
    pc = 0x1ABC
    base_address = 1000
    stride = 16

    ###### 生成 SVIs
    svi_list = svu.generate_svi(pc, base_address, stride)
    print(f"Generated SVI List: {svi_list}")

    ###### 执行 SVIs
    svu.execute_svi(svi_list)

    ###### 更新污染追踪器
    svu.update_taint_tracker(1)

    ###### 打印 SVU 的状态
    print(svu)



Generated SVI List: [(6844, 1000), (6844, 1016), (6844, 1032), (6844, 1048)]
Executed SVI for PC: 6844 at address: 1000
Executed SVI for PC: 6844 at address: 1016
Executed SVI for PC: 6844 at address: 1032
Executed SVI for PC: 6844 at address: 1048
Register 1 marked as tainted.
Scalar-Vector Unit (SVU) State:
SRF Entries:
PC: 6844, Data: 1048
Taint Tracker: {1: True}


SVR!

In [18]:
class ScalarVectorRunahead:
    def __init__(self, max_entries=32, scalar_vector_length=16):
        # 初始化各个模块
        self.stride_detector = StrideDetector(max_entries=max_entries)  # 步长检测器
        self.taint_tracker = TaintTracker(max_entries=max_entries)  # 污染追踪器
        self.hslr = HSLR()  # 头步长加载寄存器
        self.svu = ScalarVectorUnit(scalar_vector_length)  # 标量向量单元
        self.scoreboard = Scoreboard(max_entries=max_entries, max_scalars=scalar_vector_length)  # Scoreboard

    def run_svr(self, instructions):
        """
        运行 SVR 系统，根据给定的指令流进行标量向量化操作。
        :param instructions: List[Dict] 指令流，每个指令包含 PC、地址、寄存器等信息
        """
        for instruction in instructions:
            pc = instruction['pc']
            address = instruction['address']
            reg = instruction['destination_register']
            
            # 1. 步长检测 (SD)
            stride_detected, stride_confidence = self.stride_detector.detect_stride(pc, address)
            if stride_detected:
                # 检测到步长模式，更新HSLR，并生成标量向量指令（SVI）
                self.hslr.write(pc, mask=0xFFFF)  # 更新 HSLR 寄存器
                svi_list = self.svu.generate_svi(pc, base_address=address, stride=self.stride_detector.entries[pc].stride)
                print(f"Stride detected for PC {hex(pc)}: Generating SVIs {svi_list}")

                # 2. 标量向量单元 (SVU) 执行生成的 SVI
                self.svu.execute_svi(svi_list)

                # 3. 更新 Scoreboard
                self.scoreboard.add_entry(pc, status=1)  # 标记该步长指令已进入 SVR 模式
                for _ in svi_list:
                    self.scoreboard.decrement_return_counter(pc)  # 逐个减少返回计数器

            # 4. 污染追踪 (TT)
            if self.taint_tracker.is_tainted(reg):
                print(f"Register {reg} is tainted and part of an indirect chain. Updating dependencies.")
            else:
                # 将寄存器标记为污染状态
                self.taint_tracker.taint_register(register_id=reg, srf_id=1, offset=0)

            # 5. 检查指令是否已经完成执行
            if self.scoreboard.get_entry(pc) and self.scoreboard.get_entry(pc).is_completed():
                print(f"All scalar instructions for PC {hex(pc)} have completed. Clearing Scoreboard entry.")
                self.scoreboard.clear_entry(pc)

    def __repr__(self):
        return (f"Scalar Vector Runahead State:\n"
                f"Stride Detector: {self.stride_detector}\n"
                f"Taint Tracker: {self.taint_tracker}\n"
                f"HSLR: {self.hslr}\n"
                f"SVU: {self.svu}\n"
                f"Scoreboard: {self.scoreboard}")



Stride detected for PC 6844: Generating SVIs [(6844, 1032), (6844, 1048), (6844, 1064), (6844, 1080)]
Executed SVI for PC: 6844 at address: 1032
Executed SVI for PC: 6844 at address: 1048
Executed SVI for PC: 6844 at address: 1064
Executed SVI for PC: 6844 at address: 1080
Added Scoreboard Entry for PC 6844 with Return Counter set to 4
Updated Return Counter for PC 6844 by decrementing 1
Updated Return Counter for PC 6844 by decrementing 1
Updated Return Counter for PC 6844 by decrementing 1
Updated Return Counter for PC 6844 by decrementing 1
All scalar instructions for PC 6844 have completed.
All scalar instructions for PC 6844 have completed. Clearing Scoreboard entry.
Cleared Scoreboard Entry for PC 6844
Stride detected for PC 6844: Generating SVIs [(6844, 1048), (6844, 1064), (6844, 1080), (6844, 1096)]
Executed SVI for PC: 6844 at address: 1048
Executed SVI for PC: 6844 at address: 1064
Executed SVI for PC: 6844 at address: 1080
Executed SVI for PC: 6844 at address: 1096
Added Sc